In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision import transforms, models
from PIL import Image
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget

# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 载入最优模型
model_path = '/root/datasets/best_model.pth'
model = models.resnet50(weights=None)
model.fc = nn.Linear(model.fc.in_features, 3)
model.load_state_dict(torch.load(model_path))
model.eval().to(device)

# 定义Grad-CAM
target_layers = [model.layer4[-1]]

def preprocess_image_for_model(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)
    return image

def generate_grad_cam(image_path, target_class=None, output_dir='/root/datasets/良性'):
    image = preprocess_image_for_model(image_path)
    targets = [ClassifierOutputTarget(target_class)]
    cam = GradCAM(model=model, target_layers=target_layers)
    grayscale_cam = cam(input_tensor=image, targets=targets)
    grayscale_cam = grayscale_cam[0, :]
    
    # 调整原图像大小以匹配热图大小
    original_image = Image.open(image_path).convert('RGB')
    original_image_resized = original_image.resize((224, 224))
    original_image_resized = np.array(original_image_resized) / 255.0
    
    cam_image = show_cam_on_image(original_image_resized, grayscale_cam, use_rgb=True)
    
    # 获取预测结果和概率
    model.eval()
    with torch.no_grad():
        output = model(image)
        probabilities = torch.nn.functional.softmax(output[0], dim=0)
        predicted_class = probabilities.argmax().item()
        predicted_prob = probabilities[predicted_class].item()
    
    # 在图像上添加预测结果和概率
    fig, ax = plt.subplots()
    ax.imshow(cam_image)
    ax.axis('off')
    text = f'Predicted: {predicted_class} ({predicted_prob:.2f})'
    plt.text(10, 30, text, bbox=dict(facecolor='white', alpha=0.8))
    
    # 保存Grad-CAM结果
    grad_cam_output_path = os.path.join(output_dir, f'grad_cam_{os.path.basename(image_path)}')
    fig.savefig(grad_cam_output_path)
    plt.close(fig)
    
    # 清理CUDA缓存
    del image, targets, cam, grayscale_cam
    torch.cuda.empty_cache()

# 示例：生成多张图像的Grad-CAM热图
image_paths = [
    '/root/datasets/良性/1.jpeg',
    '/root/datasets/良性/2.jpeg',
    '/root/datasets/良性/3.jpeg',
    '/root/datasets/良性/4.jpeg',
    '/root/datasets/良性/5.jpeg',
    '/root/datasets/良性/6.jpeg',
    '/root/datasets/良性/7.jpeg',
    '/root/datasets/良性/8.jpeg',
    '/root/datasets/良性/9.jpeg',
    '/root/datasets/良性/10.jpeg',
    '/root/datasets/良性/11.jpeg'
]

for image_path in image_paths:
    print(f"Processing {image_path}")
    generate_grad_cam(image_path, target_class=0)  # 替换为实际的类别索引

    # 清理缓存和变量
    torch.cuda.empty_cache()


In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision import transforms, models
from PIL import Image
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget

# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 载入最优模型
model_path = '/root/datasets/best_model.pth'
model = models.resnet50(weights=None)
model.fc = nn.Linear(model.fc.in_features, 3)
model.load_state_dict(torch.load(model_path))
model.eval().to(device)

# 定义Grad-CAM
target_layers = [model.layer4[-1]]

def preprocess_image_for_model(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)
    return image

def generate_grad_cam(image_path, target_class=None, output_dir='/root/datasets/恶性'):
    image = preprocess_image_for_model(image_path)
    targets = [ClassifierOutputTarget(target_class)]
    cam = GradCAM(model=model, target_layers=target_layers)
    grayscale_cam = cam(input_tensor=image, targets=targets)
    grayscale_cam = grayscale_cam[0, :]
    
    # 调整原图像大小以匹配热图大小
    original_image = Image.open(image_path).convert('RGB')
    original_image_resized = original_image.resize((224, 224))
    original_image_resized = np.array(original_image_resized) / 255.0
    
    cam_image = show_cam_on_image(original_image_resized, grayscale_cam, use_rgb=True)
    
    # 获取预测结果和概率
    model.eval()
    with torch.no_grad():
        output = model(image)
        probabilities = torch.nn.functional.softmax(output[0], dim=0)
        predicted_class = probabilities.argmax().item()
        predicted_prob = probabilities[predicted_class].item()
    
    # 在图像上添加预测结果和概率
    fig, ax = plt.subplots()
    ax.imshow(cam_image)
    ax.axis('off')
    text = f'Predicted: {predicted_class} ({predicted_prob:.2f})'
    plt.text(10, 30, text, bbox=dict(facecolor='white', alpha=0.8))
    
    # 保存Grad-CAM结果
    grad_cam_output_path = os.path.join(output_dir, f'grad_cam_{os.path.basename(image_path)}')
    fig.savefig(grad_cam_output_path)
    plt.close(fig)
    
    # 清理CUDA缓存
    del image, targets, cam, grayscale_cam
    torch.cuda.empty_cache()

# 示例：生成多张图像的Grad-CAM热图
image_paths = [
    '/root/datasets/恶性/1.jpeg',
    '/root/datasets/恶性/2.jpeg',
    '/root/datasets/恶性/3.jpeg',
    '/root/datasets/恶性/4.jpeg',
    '/root/datasets/恶性/5.jpeg',
    '/root/datasets/恶性/6.jpeg',
    '/root/datasets/恶性/7.jpeg',
    '/root/datasets/恶性/8.jpeg',
    '/root/datasets/恶性/9.jpeg',
    '/root/datasets/恶性/10.jpeg',
    '/root/datasets/恶性/11.jpeg'
]

for image_path in image_paths:
    print(f"Processing {image_path}")
    generate_grad_cam(image_path, target_class=0)  # 替换为实际的类别索引

    # 清理缓存和变量
    torch.cuda.empty_cache()


RuntimeError: Error(s) in loading state_dict for ResNet:
	Missing key(s) in state_dict: "conv1.weight", "bn1.weight", "bn1.bias", "bn1.running_mean", "bn1.running_var", "layer1.0.conv1.weight", "layer1.0.bn1.weight", "layer1.0.bn1.bias", "layer1.0.bn1.running_mean", "layer1.0.bn1.running_var", "layer1.0.conv2.weight", "layer1.0.bn2.weight", "layer1.0.bn2.bias", "layer1.0.bn2.running_mean", "layer1.0.bn2.running_var", "layer1.0.conv3.weight", "layer1.0.bn3.weight", "layer1.0.bn3.bias", "layer1.0.bn3.running_mean", "layer1.0.bn3.running_var", "layer1.0.downsample.0.weight", "layer1.0.downsample.1.weight", "layer1.0.downsample.1.bias", "layer1.0.downsample.1.running_mean", "layer1.0.downsample.1.running_var", "layer1.1.conv1.weight", "layer1.1.bn1.weight", "layer1.1.bn1.bias", "layer1.1.bn1.running_mean", "layer1.1.bn1.running_var", "layer1.1.conv2.weight", "layer1.1.bn2.weight", "layer1.1.bn2.bias", "layer1.1.bn2.running_mean", "layer1.1.bn2.running_var", "layer1.1.conv3.weight", "layer1

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision import transforms, models
from PIL import Image
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget

# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 数据预处理
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 载入最优模型
best_model_path = '/root/datasets/ResNet直接骨3分类/best_model.pth'
model = models.resnet50(weights=None)
model.fc = nn.Linear(model.fc.in_features, 3)
checkpoint = torch.load(best_model_path, map_location=device)
model.load_state_dict(checkpoint)
model.eval().to(device)

# 定义Grad-CAM
target_layers = [model.layer4[-1]]

def preprocess_image_for_model(image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)
    return image

def generate_grad_cam(image_path, target_class=None, output_dir='/root/datasets/恶性'):
    image = preprocess_image_for_model(image_path)
    targets = [ClassifierOutputTarget(target_class)]
    cam = GradCAM(model=model, target_layers=target_layers)
    grayscale_cam = cam(input_tensor=image, targets=targets)
    grayscale_cam = grayscale_cam[0, :]
    
    # 调整原图像大小以匹配热图大小
    original_image = Image.open(image_path).convert('RGB')
    original_image_resized = original_image.resize((224, 224))
    original_image_resized = np.array(original_image_resized) / 255.0
    
    cam_image = show_cam_on_image(original_image_resized, grayscale_cam, use_rgb=True)
    
    # 获取预测结果和概率
    model.eval()
    with torch.no_grad():
        output = model(image)
        probabilities = torch.nn.functional.softmax(output[0], dim=0)
        predicted_class = probabilities.argmax().item()
        predicted_prob = probabilities[predicted_class].item()
    
    # 在图像上添加预测结果和概率
    fig, ax = plt.subplots()
    ax.imshow(cam_image)
    ax.axis('off')
    text = f'Predicted: {predicted_class} ({predicted_prob:.2f})'
    plt.text(10, 30, text, bbox=dict(facecolor='white', alpha=0.8))
    
    # 保存Grad-CAM结果
    grad_cam_output_path = os.path.join(output_dir, f'grad_cam_{os.path.basename(image_path)}')
    fig.savefig(grad_cam_output_path)
    plt.close(fig)
    
    # 清理CUDA缓存
    del image, targets, cam, grayscale_cam
    torch.cuda.empty_cache()

# 示例：生成多张图像的Grad-CAM热图
image_paths = [
    '/root/datasets/恶性/1.jpeg',
    '/root/datasets/恶性/2.jpeg',
    '/root/datasets/恶性/3.jpeg',
    '/root/datasets/恶性/4.jpeg',
    '/root/datasets/恶性/5.jpeg',
    '/root/datasets/恶性/6.jpeg',
    '/root/datasets/恶性/7.jpeg',
    '/root/datasets/恶性/8.jpeg',
    '/root/datasets/恶性/9.jpeg',
    '/root/datasets/恶性/10.jpeg',
    '/root/datasets/恶性/11.jpeg'
]

for image_path in image_paths:
    print(f"Processing {image_path}")
    generate_grad_cam(image_path, target_class=0)  # 替换为实际的类别索引

    # 清理缓存和变量
    torch.cuda.empty_cache()


Processing /root/datasets/恶性/1.jpeg
Processing /root/datasets/恶性/2.jpeg
Processing /root/datasets/恶性/3.jpeg
Processing /root/datasets/恶性/4.jpeg
Processing /root/datasets/恶性/5.jpeg
Processing /root/datasets/恶性/6.jpeg
Processing /root/datasets/恶性/7.jpeg
Processing /root/datasets/恶性/8.jpeg
Processing /root/datasets/恶性/9.jpeg
Processing /root/datasets/恶性/10.jpeg
Processing /root/datasets/恶性/11.jpeg


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision import transforms, models
from PIL import Image
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget

# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 数据预处理
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 载入最优模型
best_model_path = '/root/datasets/ResNet直接骨3分类/best_model.pth'
model = models.resnet50(weights=None)
model.fc = nn.Linear(model.fc.in_features, 3)
checkpoint = torch.load(best_model_path, map_location=device)
model.load_state_dict(checkpoint)
model.eval().to(device)

# 定义Grad-CAM
target_layers = [model.layer4[-1]]

def preprocess_image_for_model(image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)
    return image

def generate_grad_cam(image_path, target_class=None, output_dir='/root/datasets/恶性'):
    image = preprocess_image_for_model(image_path)
    targets = [ClassifierOutputTarget(target_class)]
    cam = GradCAM(model=model, target_layers=target_layers)
    grayscale_cam = cam(input_tensor=image, targets=targets)
    grayscale_cam = grayscale_cam[0, :]
    
    # 调整原图像大小以匹配热图大小
    original_image = Image.open(image_path).convert('RGB')
    original_image_resized = original_image.resize((224, 224))
    original_image_resized = np.array(original_image_resized) / 255.0
    
    cam_image = show_cam_on_image(original_image_resized, grayscale_cam, use_rgb=True)
    
    # 获取预测结果和概率
    model.eval()
    with torch.no_grad():
        output = model(image)
        probabilities = torch.nn.functional.softmax(output[0], dim=0)
        predicted_class = probabilities.argmax().item()
        predicted_prob = probabilities[predicted_class].item()
    
    # 在图像上添加预测结果和概率
    fig, ax = plt.subplots()
    ax.imshow(cam_image)
    ax.axis('off')
    text = f'Predicted: {predicted_class} ({predicted_prob:.2f})'
    plt.text(10, 30, text, bbox=dict(facecolor='white', alpha=0.8))
    
    # 保存Grad-CAM结果
    grad_cam_output_path = os.path.join(output_dir, f'grad_cam_{os.path.basename(image_path)}')
    fig.savefig(grad_cam_output_path)
    plt.close(fig)
    
    # 清理CUDA缓存
    del image, targets, cam, grayscale_cam
    torch.cuda.empty_cache()

# 示例：生成多张图像的Grad-CAM热图
image_paths = [
    '/root/datasets/良性/1.jpeg',
    '/root/datasets/良性/2.jpeg',
    '/root/datasets/良性/3.jpeg',
    '/root/datasets/良性/4.jpeg',
    '/root/datasets/良性/5.jpeg',
    '/root/datasets/良性/6.jpeg',
    '/root/datasets/良性/7.jpeg',
    '/root/datasets/良性/8.jpeg',
    '/root/datasets/良性/9.jpeg',
    '/root/datasets/良性/10.jpeg',
    '/root/datasets/良性/11.jpeg'
]
for image_path in image_paths:
    print(f"Processing {image_path}")
    generate_grad_cam(image_path, target_class=0)  # 替换为实际的类别索引

    # 清理缓存和变量
    torch.cuda.empty_cache()
